In [1]:
import itertools
import torch

import sys
sys.path.append('../src')

from tqdm.autonotebook import tqdm

from data_utils import preprocess, make_train_valid_dfs, build_loaders
from config import default_config
from transformers import DistilBertTokenizer
from clip_utils import CLIPModel
from train_eval_utils import train_epoch, valid_epoch

In [2]:
config = default_config

In [3]:
# preprocess(config['raw_file_path'], 0)

In [4]:
train_df, valid_df = make_train_valid_dfs(config["clean_file_path"], 0.8)

In [5]:
tokenizer = DistilBertTokenizer.from_pretrained(config['text_tokenizer'])
train_loader = build_loaders(train_df, tokenizer, mode="train", config=config)

In [6]:
model = CLIPModel(config)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
params = [
    {"params": model.image_encoder.parameters(), "lr": config['image_encoder_lr']},
    {"params": model.text_encoder.parameters(), "lr": config['text_encoder_lr']},
    {"params": itertools.chain(
        model.image_projection.parameters(), model.text_projection.parameters()
    ), "lr": config['projection_head_lr'], "weight_decay": config["weight_decay"]}
]
optimizer = torch.optim.AdamW(params, weight_decay=0.)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="min", patience=config['patience'], factor=config['factor']
)
step = "epoch"
device = config['device']

best_loss = float('inf')
for epoch in range(config['epochs']):
    print(f"Epoch: {epoch + 1}")
    model.train()
    train_loss = train_epoch(model, train_loader, optimizer, lr_scheduler, step, device)
    model.eval()
    with torch.no_grad():
        valid_loss = valid_epoch(model, valid_loader, device)

    if valid_loss.avg < best_loss:
        best_loss = valid_loss.avg
        # torch.save(model.state_dict(), "best.pt")
        # print("Saved Best Model!")
        print(f"{best_loss=}")

    lr_scheduler.step(valid_loss.avg)

Epoch: 1


  0%|          | 0/1012 [00:00<?, ?it/s]

[W NNPACK.cpp:51] Could not initialize NNPACK! Reason: Unsupported hardware.


KeyboardInterrupt: 